In [1]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
import pandas as pd

/root/anaconda3/envs/tensorflow_cpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
root_path = "../"
path_data_clean = root_path + "data/clean/"
dfNoticias = pd.read_pickle(path_data_clean + "/dfNoticiasCleanV2.p")


dfNoticias.info()

raw_text = ""

count = 0
for index, row in dfNoticias.iterrows():    
    if index < 700:
        raw_text += row["Cuerpo"]
        
# load ascii text and covert to lowercase
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351681 entries, 0 to 367147
Data columns (total 16 columns):
Cuerpo       351681 non-null object
Fecha        351681 non-null object
Hora         351681 non-null object
ID           351681 non-null object
Resumen      350517 non-null object
Seccion_1    351681 non-null object
Seccion_2    187037 non-null object
Seccion_3    59174 non-null object
Subtema_1    255959 non-null object
Subtema_2    114189 non-null object
Subtema_3    39513 non-null object
Tema_1       349253 non-null object
Tema_2       179833 non-null object
Tema_3       57572 non-null object
Titular      351681 non-null object
Type         351680 non-null object
dtypes: object(16)
memory usage: 45.6+ MB


In [3]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Characters:  1118101
Total Vocab:  90
Total Patterns:  1118001


In [4]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model

# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))

# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))



model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')



In [8]:
import numpy as np

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [15]:
# load the network weights
#filename = "weights-improvement-03-2.6103-bigger.hdf5"

filename = "weights-improvement-26-1.4667-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
#pattern = "s álvaro elizalde y javiera blanco continuarán en sus cargos y que se sumarán los parlamentarios ele"
print ("Semilla :")
print ( "\"", ''.join([int_to_char[value] for value in pattern]), "\"" )
print ( "\n \n \n")

# generate characters
for i in range(200):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = sample(prediction[0],0.5)
    #index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla :
" o, que hay poco interés por ir a votar.
la encuesta también demostró que un 44,5 por ciento de las p "

 
 

ropios fuertes de la información

/root/anaconda3/envs/tensorflow_cpu/lib/python3.5/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


 del acuor de cuitta está a asrorar a los menores de la policía de la presidenta la disección de la indorsric del ministerio de es

KeyboardInterrupt: 

array([[2.10725120e-05, 2.83720437e-03, 1.19657442e-12, 4.44530742e-05,
        2.58455388e-12, 5.24282614e-12, 1.13328292e-12, 1.86961189e-12,
        2.15690739e-06, 9.54890402e-06, 1.07196711e-05, 2.64692863e-13,
        1.29307697e-13, 9.22367180e-05, 3.28438218e-05, 7.31752807e-05,
        2.75751351e-08, 1.17182644e-07, 2.95644031e-06, 8.76455147e-07,
        1.52448948e-07, 1.49691759e-09, 1.48645596e-09, 1.37598908e-11,
        5.04607779e-11, 2.48228060e-09, 5.97353005e-13, 6.14839064e-06,
        5.89342699e-06, 4.85994917e-07, 1.45128229e-10, 2.22773383e-10,
        1.20424809e-10, 4.94142114e-08, 1.63946766e-03, 7.14836922e-03,
        1.71167389e-01, 2.45517567e-02, 9.17841494e-02, 5.51075675e-04,
        2.28669748e-01, 1.80864357e-03, 1.00474069e-02, 1.39759816e-02,
        1.63497252e-03, 1.53239882e-02, 4.09882748e-03, 2.26581231e-01,
        4.43256972e-03, 1.16385520e-03, 1.35810266e-03, 2.18958892e-02,
        8.68804306e-02, 4.19229874e-03, 5.64447022e-04, 1.865551